In [ ]:
from matplotlib import animation
from matplotlib.animation import PillowWriter
import matplotlib.pyplot as plt
import numpy as np


class OOPlanet:
    def __init__(self, rad, pomg):
        self.rad = rad
        self.pomg = pomg
        
    def getX(self, time):
        return self.rad * np.cos(self.pomg * time)
    
    def getY(self, time):
        return self.rad * np.sin(self.pomg * time)
    
    def position(self, time):
        return (self.getX(time), self.getY(time))
        
    def evolve(self, start, end, resolution):
        tinterval = end - start
        timestep = tinterval / resolution
        xarray = []
        yarray = []
        t = start
        for i in range(resolution):
            xarray.append(self.position(t)[0]
            yarray.append(self.position(t)[1])
            t = ((i+1)*timestep) + t
        return [xarray, yarray]

class OOPmoon(OOPlanet):
    def __init__(self, rad, pomg):
        super().__init__(rad, pomg)
                        
    def position(self, time):
        xs = self.rad * np.cos(self.pomg * time) + super().getX(time)
        ys = self.rad * np.sin(self.pomg * time) + super().getY(time)
        return (xs, ys)
    

    def evolve(self, start, end, resolution):
        tinterval = end - start
        timestep = tinterval / resolution
        xarray = []
        yarray = []
        t = start
        for i in range(resolution):
            xarray.append(self.position(t)[0])
            yarray.append(self.position(t)[1])
            t = ((i+1)*timestep) + t
        return [xarray, yarray]